In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parse backtest result

In [23]:
import json

file_name = '../backtest_results/backtest-20220418-182507.json'

with open(file_name) as f:
    json_file = json.load(f)

In [24]:
start_balance = json_file['balance'][0]
start_balance

{'date': '2021-04-17 17:00:00',
 'XRP': 0.0,
 'DOT': 1.69,
 'BTC': 0.00143,
 'ETH': 0.0171,
 'ADA': 0.0,
 'BNB': 0.09669333,
 'AVAX': 0.16,
 'SOL': 0.71}

In [25]:
final_balance = json_file['balance'][-1]
final_balance

{'date': '2022-04-17 17:00:00',
 'XRP': 18.075999999999993,
 'DOT': 10.853000000000002,
 'BTC': 0.0013980800000000003,
 'ETH': 0.0052,
 'ADA': 0.0,
 'BNB': 0.30862133,
 'AVAX': 0.23158999999999885,
 'SOL': 0.1289300000000002,
 'USDT': 0.12026464389987268}

In [26]:
from prettytable import *

msg = PrettyTable()
msg.field_names = ["Coin", "From", "To", "Growth"]

for key in final_balance:
    end = final_balance[key]
    start = start_balance[key] if key in start_balance else '0.0'
    growth = ''
    if key != 'date':
        s = float(start)
        e = float(end)
        if s > 0:
            growth = round(((e - s) / s) * 100, 2)
        else:
            growth = 'inf'
    msg.add_row([
        key,
        start,
        end,
        growth
    ])

msg.align = "l"
print(msg)

+------+---------------------+-----------------------+--------+
| Coin | From                | To                    | Growth |
+------+---------------------+-----------------------+--------+
| date | 2021-04-17 17:00:00 | 2022-04-17 17:00:00   |        |
| XRP  | 0.0                 | 18.075999999999993    | inf    |
| DOT  | 1.69                | 10.853000000000002    | 542.19 |
| BTC  | 0.00143             | 0.0013980800000000003 | -2.23  |
| ETH  | 0.0171              | 0.0052                | -69.59 |
| ADA  | 0.0                 | 0.0                   | inf    |
| BNB  | 0.09669333          | 0.30862133            | 219.18 |
| AVAX | 0.16                | 0.23158999999999885   | 44.74  |
| SOL  | 0.71                | 0.1289300000000002    | -81.84 |
| USDT | 0.0                 | 0.12026464389987268   | inf    |
+------+---------------------+-----------------------+--------+


In [27]:
from sqlitedict import SqliteDict
from dateutil import parser
from datetime import timedelta

sqlite_cache = SqliteDict("../data/backtest_cache.db")

values = {}
start_date = parser.parse(start_balance['date'])
starting_value_in_usdt = 0
for k in start_balance:
    if k != 'date':
        v = float(start_balance[k])
        if k == 'USDT':
            starting_value_in_usdt += v
        else:
            target_date = start_date.strftime("%d %b %Y %H:%M:%S")
            key = f"{k}USDT - {target_date}"
            usdt_val = sqlite_cache.get(key, None)
            starting_value_in_usdt += usdt_val * v
            values[k] = {
                'start': v,
                'start_price': usdt_val,
                'start_value': usdt_val * v
            }

final_date = parser.parse(final_balance['date'])
final_date = final_date - timedelta(days=1)
finaling_value_in_usdt = 0
for k in final_balance:
    if k != 'date':
        v = float(final_balance[k])
        if k == 'USDT':
            finaling_value_in_usdt += v
        else:
            target_date = final_date.strftime("%d %b %Y %H:%M:%S")
            key = f"{k}USDT - {target_date}"
            usdt_val = sqlite_cache.get(key, None)
            #print(f'{key} :\tusdt_val * v = {usdt_val * v}')
            finaling_value_in_usdt += usdt_val * v
            values[k] = {
                **values[k],
                **{
                    'final': v,
                    'final_price': usdt_val,
                    'final_value': usdt_val * v
                }
            }
print(f'{starting_value_in_usdt} USDT -> {finaling_value_in_usdt} USDT')

msg = PrettyTable()
msg.field_names = ['coin', 'start_balance', 'final_balance', 'start_price', 'final_price', 'start_value', 'final_value']

for k in values:
    row = values[k]
    price_growth = ((row['final_price'] - row['start_price']) / row['start_price']) * 100 if row[
                                                                                              'start_price'] > 0 else 0
    value_growth = ((row['final_value'] - row['start_value']) / row['start_value']) * 100 if row[
                                                                                                 'start_value'] > 0 else 0
    msg.add_row(
        [
            k,
            row['start'],
            row['final'],

            row['start_price'],
            f"{row['final_price']}({price_growth}%)",

            row['start_value'],
            f"{row['final_value']}({value_growth}%)",

        ]
    )
msg.align = "l"
print(msg)


273.860752096477 USDT -> 445.8734321702999 USDT
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+------------------------------------------+
| coin | start_balance | final_balance         | start_price | final_price                    | start_value        | final_value                              |
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+------------------------------------------+
| XRP  | 0.0           | 18.075999999999993    | 1.57105     | 0.7719(-50.86725438401069%)    | 0.0                | 13.952864399999996(0%)                   |
| DOT  | 1.69          | 10.853000000000002    | 43.3326     | 18.53(-57.2377378694101%)      | 73.232094          | 201.10609000000005(174.6146928421848%)   |
| BTC  | 0.00143       | 0.0013980800000000003 | 60377.01    | 40203.33(-33.412850354795644%) | 86.33912430000001  | 56.20747160640001(-34.8991872895333

In [28]:
'''

273.860752096477 USDT -> 730.3140972761008 USDT
+------+---------------+------------------------+-------------+--------------------------------+--------------------+------------------------------------------+
| coin | start_balance | final_balance          | start_price | final_price                    | start_value        | final_value                              |
+------+---------------+------------------------+-------------+--------------------------------+--------------------+------------------------------------------+
| XRP  | 0.0           | 0.93100000000004       | 1.57105     | 0.7719(-50.86725438401069%)    | 0.0                | 0.718638900000031(0%)                    |
| DOT  | 1.69          | 39.268209999999996     | 43.3326     | 18.53(-57.2377378694101%)      | 73.232094          | 727.6399313(893.6079819047643%)          |
| BTC  | 0.00143       | 7.400000000001156e-06  | 60377.01    | 40203.33(-33.412850354795644%) | 86.33912430000001  | 0.29750464200004645(-99.65542314169609%) |
| ETH  | 0.0171        | 0.0                    | 2350.71     | 3026.17(28.73429729741227%)    | 40.197141          | 0.0(-100.0%)                             |
| ADA  | 0.0           | 0.0                    | 1.382       | 0.944(-31.693198263386396%)    | 0.0                | 0.0(0%)                                  |
| BNB  | 0.09669333    | 0.00030733000000005006 | 516.2569    | 414.0(-19.807367223566402%)    | 49.91859879647699  | 0.12723462000002073(-99.74511580239108%) |
| AVAX | 0.16          | 0.008200000000005758   | 34.4838     | 77.12(123.64124603436977%)     | 5.5174080000000005 | 0.6323840000004441(-88.5383861407305%)   |
| SOL  | 0.71          | 0.008459999999999468   | 26.2766     | 101.77(287.3027712870006%)     | 18.656385999999998 | 0.8609741999999458(-95.38509655621435%)  |
+------+---------------+------------------------+-------------+--------------------------------+--------------------+------------------------------------------+


273.860752096477 USDT -> 467.7091450441003 USDT
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+-----------------------------------------+
| coin | start_balance | final_balance         | start_price | final_price                    | start_value        | final_value                             |
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+-----------------------------------------+
| XRP  | 0.0           | 16.158000000000015    | 1.57105     | 0.7719(-50.86725438401069%)    | 0.0                | 12.472360200000013(0%)                  |
| DOT  | 1.69          | 9.68194               | 43.3326     | 18.53(-57.2377378694101%)      | 73.232094          | 179.40634820000002(144.9832285281915%)  |
| BTC  | 0.00143       | 0.0005963300000000008 | 60377.01    | 40203.33(-33.412850354795644%) | 86.33912430000001  | 23.974451778900033(-72.23222730914352%) |
| ETH  | 0.0171        | 0.0026                | 2350.71     | 3026.17(28.73429729741227%)    | 40.197141          | 7.868042(-80.42636415360982%)           |
| ADA  | 0.0           | 0.0                   | 1.382       | 0.944(-31.693198263386396%)    | 0.0                | 0.0(0%)                                 |
| BNB  | 0.09669333    | 0.5096593300000003    | 516.2569    | 414.0(-19.807367223566402%)    | 49.91859879647699  | 210.99896262000013(322.6860683334953%)  |
| AVAX | 0.16          | 0.2313200000000002    | 34.4838     | 77.12(123.64124603436977%)     | 5.5174080000000005 | 17.839398400000015(223.32933145419034%) |
| SOL  | 0.71          | 0.14870000000000005   | 26.2766     | 101.77(287.3027712870006%)     | 18.656385999999998 | 15.133199000000005(-18.88461677411688%) |
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+-----------------------------------------+


273.860752096477 USDT -> 445.8734321702999 USDT
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+------------------------------------------+
| coin | start_balance | final_balance         | start_price | final_price                    | start_value        | final_value                              |
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+------------------------------------------+
| XRP  | 0.0           | 18.075999999999993    | 1.57105     | 0.7719(-50.86725438401069%)    | 0.0                | 13.952864399999996(0%)                   |
| DOT  | 1.69          | 10.853000000000002    | 43.3326     | 18.53(-57.2377378694101%)      | 73.232094          | 201.10609000000005(174.6146928421848%)   |
| BTC  | 0.00143       | 0.0013980800000000003 | 60377.01    | 40203.33(-33.412850354795644%) | 86.33912430000001  | 56.20747160640001(-34.89918728953334%)   |
| ETH  | 0.0171        | 0.0052                | 2350.71     | 3026.17(28.73429729741227%)    | 40.197141          | 15.736084(-60.852728307219664%)          |
| ADA  | 0.0           | 0.0                   | 1.382       | 0.944(-31.693198263386396%)    | 0.0                | 0.0(0%)                                  |
| BNB  | 0.09669333    | 0.30862133            | 516.2569    | 414.0(-19.807367223566402%)    | 49.91859879647699  | 127.76923062000002(155.95516240535449%)  |
| AVAX | 0.16          | 0.23158999999999885   | 34.4838     | 77.12(123.64124603436977%)     | 5.5174080000000005 | 17.86022079999991(223.70672605687147%)   |
| SOL  | 0.71          | 0.1289300000000002    | 26.2766     | 101.77(287.3027712870006%)     | 18.656385999999998 | 13.121206100000022(-29.669089715446372%) |
+------+---------------+-----------------------+-------------+--------------------------------+--------------------+------------------------------------------+

'''

'\n\n273.860752096477 USDT -> 730.3140972761008 USDT\n+------+---------------+------------------------+-------------+--------------------------------+--------------------+------------------------------------------+\n| coin | start_balance | final_balance          | start_price | final_price                    | start_value        | final_value                              |\n+------+---------------+------------------------+-------------+--------------------------------+--------------------+------------------------------------------+\n| XRP  | 0.0           | 0.93100000000004       | 1.57105     | 0.7719(-50.86725438401069%)    | 0.0                | 0.718638900000031(0%)                    |\n| DOT  | 1.69          | 39.268209999999996     | 43.3326     | 18.53(-57.2377378694101%)      | 73.232094          | 727.6399313(893.6079819047643%)          |\n| BTC  | 0.00143       | 7.400000000001156e-06  | 60377.01    | 40203.33(-33.412850354795644%) | 86.33912430000001  | 0.2975046420000464